In [41]:
!pip install wget
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np
import wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=1ad6dce280aaa6496034281c30b60af650c34878d62a873d4abf03c6d1ed4446
  Stored in directory: c:\users\clarence.toh\appdata\local\pip\cache\wheels\04\5f\3e\46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [44]:
# Read the airline data into pandas dataframe
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
spacex_df=pd.read_csv(spacex_csv_file)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [45]:
def success_categorise(row):  
    if row['class'] == 0:
        return 'No'
    else:
        return 'Yes'

In [46]:
# Create a dash application
app = dash.Dash(__name__)

In [47]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                        html.Div(dcc.Dropdown(id='site-dropdown', 
                                                     # Update dropdown values using list comphrehension
                                                     options=[{'label': 'All Sites', 'value': 'ALL'},
                                                              {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                              {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                              {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                              {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},],
                                                     value = 'ALL',
                                                     placeholder = "Select a Launch Site here",
                                                     searchable = True
                                                     )),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                #html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       100: '100'},
                                                value=[min_payload, max_payload]),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                
                                ])

In [48]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if(entered_site) == "ALL":
        filtered_df = spacex_df[spacex_df['class'] != 0.0]
        chart_title = "Total Success Launches by Site"
        chart_names = "Launch Site"
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] != entered_site]
        filtered_df['Success'] = filtered_df.apply(lambda row: success_categorise(row), axis=1)
        filtered_df = filtered_df.groupby(["Success"])["class"].count().reset_index()
        chart_title = (f"Total Success Launches for {entered_site}")
        chart_names = "Success"
        
    fig = px.pie(filtered_df, values='class', 
    names = chart_names, 
    title = chart_title)
    
    return fig



        
        # return the outcomes piechart for a selected site

In [49]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])

def get_scatter_chart(entered_site, slider):
    
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] > slider[0]) & (spacex_df['Payload Mass (kg)'] < slider[1])]
    if(entered_site) == "ALL":
        filtered_df = filtered_df
        chart_title = "Correlation betwen Payload and Success for all Sites"

    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] != entered_site]
        chart_title = f"Correlation betwen Payload and Success for {entered_site}"
        
    fig = px.scatter(data_frame = filtered_df,
                     x = 'Payload Mass (kg)',
                     y = 'class',
                     color="Booster Version Category",
                     title = chart_title)
    
    
    
    return fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
